In [3]:
import matplotlib.pyplot as plt
import numpy as np
import math as m
import os
import datetime
import pandas as pd

In [48]:
# Read the log file - extract log_times and log_h_or_c
log_file = open(os.path.join('/Users/micol/Documents/Imperial/Year4/Master_Project/bruker_logs', 'EventLog_20180927112225.log')).readlines()

log_file_list = []
for i in log_file:
    log_file_list.append(i.split(' '))
del log_file_list[0:9]    
for i in log_file_list:
    i.pop(5)
    i.pop(4)
    i.pop(3)
    i.pop(0)
    i[0]=' '.join(i[0:1])
print(log_file_list) # each item has length either 2 or 3. Length 3 is at start of h/c BB measurement

[['12:23:01', 'PM', 'start:', 'h0\n'], ['12:23:03', 'PM', '3\n'], ['12:23:05', 'PM', '3\n'], ['12:23:08', 'PM', '3\n'], ['12:23:10', 'PM', '3\n'], ['12:23:12', 'PM', '3\n'], ['12:23:15', 'PM', '3\n'], ['12:23:17', 'PM', '3\n'], ['12:23:20', 'PM', '3\n'], ['12:23:22', 'PM', '3\n'], ['12:23:24', 'PM', '3\n'], ['12:23:27', 'PM', '3\n'], ['12:23:29', 'PM', '3\n'], ['12:23:32', 'PM', '3\n'], ['12:23:34', 'PM', '3\n'], ['12:23:37', 'PM', '3\n'], ['12:23:39', 'PM', '3\n'], ['12:23:41', 'PM', '3\n'], ['12:23:43', 'PM', '3\n'], ['12:23:46', 'PM', '3\n'], ['12:23:48', 'PM', '3\n'], ['12:23:51', 'PM', '3\n'], ['12:23:53', 'PM', '3\n'], ['12:23:55', 'PM', '3\n'], ['12:23:57', 'PM', '2\n'], ['12:24:00', 'PM', '2\n'], ['12:24:00', 'PM', 'start:', 'h1\n'], ['12:24:02', 'PM', '3\n'], ['12:24:05', 'PM', '3\n'], ['12:24:07', 'PM', '3\n'], ['12:24:09', 'PM', '3\n'], ['12:24:12', 'PM', '3\n'], ['12:24:14', 'PM', '3\n'], ['12:24:16', 'PM', '3\n'], ['12:24:19', 'PM', '3\n'], ['12:24:21', 'PM', '3\n'], ['12:

In [49]:
log_times = []
log_h_c = []

for i in log_file_list:
    time = datetime.datetime.strptime(i[0]+i[1], '%I:%M:%S%p')
    log_times.append(time.second+time.minute*60+time.hour*3600)
    if len(i) == 4:
        log_h_c.append(i[-1][0])
    elif len(i) == 3:
        log_h_c.append(log_h_c[-1])
    else:
        raise Exception('Error')

print(log_times) 
print(log_h_c)

[44581, 44583, 44585, 44588, 44590, 44592, 44595, 44597, 44600, 44602, 44604, 44607, 44609, 44612, 44614, 44617, 44619, 44621, 44623, 44626, 44628, 44631, 44633, 44635, 44637, 44640, 44640, 44642, 44645, 44647, 44649, 44652, 44654, 44656, 44659, 44661, 44664, 44666, 44669, 44672, 44674, 44676, 44679, 44681, 44683, 44686, 44690, 44692, 44694, 44696, 44700, 44700, 44702, 44705, 44707, 44710, 44712, 44714, 44717, 44719, 44721, 44724, 44726, 44729, 44731, 44733, 44736, 44738, 44740, 44742, 44745, 44748, 44750, 44752, 44755, 44757, 44760, 44760, 44763, 44765, 44767, 44770, 44772, 44775, 44777, 44779, 44782, 44784, 44787, 44789, 44792, 44794, 44797, 44799, 44802, 44804, 44806, 44809, 44812, 44814, 44816, 44820, 44820, 44822, 44824, 44827, 44829, 44831, 44834, 44836, 44839, 44841, 44844, 44846, 44849, 44851, 44853, 44855, 44858, 44861, 44863, 44866, 44868, 44870, 44873, 44875, 44877, 44880, 44880, 44882, 44885, 44887, 44890, 44892, 44894, 44896, 44899, 44901, 44904, 44906, 44909, 44911, 44913

In [51]:
# create dataframe with all values? so then we can find a value by its index 
hc_log = pd.DataFrame(
    {'Time (sec since midnight)': log_times,
     'h/c': log_h_c,
    })
hc_log
#print(hc_log.loc[51])

,Time (sec since midnight),h/c
0,44581,h
1,44583,h
2,44585,h
3,44588,h
4,44590,h
5,44592,h
6,44595,h
7,44597,h
8,44600,h
9,44602,h


In [17]:
# Read log temperature file
def read_log_file_temperature(directory, filename):
    lines = open(os.path.join(directory, filename)).readlines()
    time = [str(datetime.datetime.strptime(i[0:17], '%Y%m%d,%H:%M:%S')).split(' ')[1] for i in lines]
        
        time = datetime.datetime.strptime(i[0]+i[1], '%I:%M:%S%p')
    log_times.append(time.second+time.minute*60+time.hour*3600)
    
    temp_cold = [float((i[-6:-1])) for i in lines] # temperature of external black body
    temp_hot = [float((i[-12:-7])) for i in lines] # temperature of external black body
    return temp_hot, temp_cold, time

In [18]:
temp_hot, temp_cold, time = read_log_file_temperature('/Users/micol/Documents/Imperial/Year4/Master_Project/bruker_logs', 'thursday2-20180927113645.log')

In [19]:
temp_log = pd.DataFrame(
    {'Time': time,
     'hot': temp_hot,
     'cold': temp_cold
    })
temp_log

,Time,cold,hot
0,12:36:45,22.38,64.83
1,12:36:48,22.38,64.60
2,12:36:50,22.39,64.61
3,12:36:52,22.40,64.64
4,12:36:54,22.40,64.66
5,12:36:57,22.40,64.65
6,12:36:59,22.41,64.68
7,12:37:01,22.42,64.71
8,12:37:03,22.41,64.74
9,12:37:05,22.42,64.72


In [ ]:
# try with one raw spectrum for now
def read_prn_raw(directory, filename):
    df = pd.read_table(os.path.join(directory, filename))
    arr = df.values
    wns = []
    raws = []

    for a in arr:
        wns.append(a[0])
        raws.append(a[1])
        
    time = str(filename[12:20]).split('_')    
    
    return time, wns, raws

In [ ]:
time1, wnsc1, rawsc1 = read_prn_raw('/Users/micol/Documents/Imperial/Year4/Master_Project/prn_files', '(2018_09_27_14_24_05)_Measurement1.prn')
time2, wnsc2, rawsc2 = read_prn_raw('/Users/micol/Documents/Imperial/Year4/Master_Project/prn_files', '(2018_09_27_14_24_05)_Measurement2.prn')
time3, wnsc3, rawsc3 = read_prn_raw('/Users/micol/Documents/Imperial/Year4/Master_Project/prn_files', '(2018_09_27_14_24_05)_Measurement3.prn')
time4, wnsc4, rawsc4 = read_prn_raw('/Users/micol/Documents/Imperial/Year4/Master_Project/prn_files', '(2018_09_27_14_24_05)_Measurement4.prn')

In [ ]:
time1